# CSCI 3155 : Assignment 9

__Name__: Ian McGregor

In [1923]:
// TEST HELPER: Do not EDIT this cell please.
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

passed: (points: Int)Unit


## Problem 1 (25 Points): Mutual Recursion in Lettuce

In class, we have explored recursive functions in lettuce using the _let rec_ syntax. In this problem, we will 
explore, mutually recursive function, specifically two mutually recursive functions.

Consider: 

~~~
let rec 
        pos = function (x) 
                 if (x >= 0)
                 then x
                 else neg(x)
        neg = function (y)
                 if (y <= 0)
                 then pos (1 + y * y)
                 else pos(1 - y)
   in 
     neg(10)
~~~

The two functions are _mutually recursive_ since `pos` calls `neg` and vice-versa.
Convince yourself that thhe program must return `82`.

### Problem 1a (5 points): Extending the Abstract Syntax

We wish to add a new rule for two mutually recursive functions

$$ \mathbf{Expr1}\ \rightarrow\ LetRec2( \mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr1}, \mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr1}, \mathbf{Expr1} ) $$

Such that a mutual call such as

~~~
let rec 
         f1 = function (x1) e1
         f2 = function (x2) e2
     in 
         e3
~~~
is represented in the AST as

~~~
LetRec2(f1, x1, e1, f2, x2, e2, e3)
~~~

Extend the existing AST specification to add support for `LetRec2`.

In [1924]:
sealed trait Program1
case class Main(e: Expr1) extends Program1

sealed trait Expr1
case class  Const(n : Int) extends Expr1
case class  Ident(s : String) extends Expr1
case object True extends Expr1
case object False extends Expr1
case class  Plus( e1 : Expr1, e2 : Expr1) extends Expr1
case class  Minus( e1 : Expr1, e2 : Expr1) extends Expr1
case class  Mult( e1 : Expr1, e2 : Expr1) extends Expr1
case class  Neg( e1 : Expr1) extends Expr1
case class  And( e1 : Expr1, e2 : Expr1) extends Expr1
case class  Or( e1 : Expr1, e2 : Expr1) extends Expr1
case class  Geq( e1 : Expr1, e2 : Expr1) extends Expr1
case class  Eq( e1 : Expr1, e2 : Expr1) extends Expr1
case class  ITE( ec : Expr1, et : Expr1, ef : Expr1) extends Expr1
case class  Let( name : String, ed : Expr1, ei : Expr1) extends Expr1
case class  FunDef( arg : String, eb : Expr1) extends Expr1
case class  FunCall( ef : Expr1, ea : Expr1) extends Expr1
case class  LetRec( f : String, x : String, eb : Expr1, ei : Expr1) extends Expr1
case class  LetRec2(f1 : String, x1 : String, e1 : Expr1, f2 : String, x2 : String, e2 : Expr1, e3 : Expr1) extends Expr1

defined trait Program1
defined class Main
defined trait Expr1
defined class Const
defined class Ident
defined object True
defined object False
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined class And
defined class Or
defined class Geq
defined class Eq
defined class ITE
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class LetRec2


In [1925]:
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val bar = Ident("bar")

val e1 = ITE( Geq(x, Const(0)), x, FunCall(bar, Plus(x, Const(1))))
val e2 = ITE( Geq(Const(10), x), Plus(Const(2), x), FunCall(foo, Minus(x, Const(2))))
val e3 = FunCall(bar, Const(1))
val lr2 = LetRec2("foo", "x", e1, "bar", "x", e2, e3)
val p1 = Main(lr2)
passed(2)


*** Tests Passed (2 points) ***


null

In [1926]:
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val bar = Ident("bar")
val e11 = ITE(Geq(x, Const(0)), FunCall(bar, Minus(Const(1), x)),Mult(x, FunCall(foo,  Minus(x, Const(1))) ))
val e1 = ITE(Eq(x, Const(0)), Const(1), e11)
val e2 = ITE(Geq(Const(0), y), Mult(Const(-5), y) , FunCall(foo, y))
val e3 = FunCall(foo, Const(10))
val lr3 = LetRec2("foo", "x", e1, "bar", "y", e2, e3)
passed(3)


*** Tests Passed (3 points) ***


null

### Problem 1b (10 points): Code up the Environment Spec

We how to build an environment that handles regular recursive calls using the `ExtendRec` construct.

Now we propose an `ExtendMutualRec2` construct such that

$$\begin{array}{c}
\sigma\prime\prime  = \sigma[f1 \rightarrow Closure(x1,e1, \sigma\prime\prime), f1 \rightarrow Closure(x2,e2, \sigma\prime\prime)] \qquad
\sigma\prime\prime \vdash \Downarrow v \\\
\\
\hline
\sigma \vdash \texttt{LetRec2(f1, x1, e1, f2, x2, e2, e)} \Downarrow v \\
\end{array}\text{LetRec2}$$


The mathematical specification of $\texttt{ExtendMutualRec2}$. Let $\sigma = \texttt{ExtendMutualRec2}(\pi, \texttt{f1, x1, e1, f2, x2, e2}))$ is as follows:

$$ \sigma(y) = \begin{cases}
\texttt{Closure}(x_1, e_1, \sigma) & \mbox{if}\ y = \texttt{f1} \\
\texttt{Closure}(x_2, e_2, \sigma) & \mbox{if}\ y = \texttt{f2} \\
\pi(y) & \text{otherwise}\\
\end{cases}$$



Implement the environment for `ExtendMutualRec`, and the functions ``contains`` and ``lookupEnv``.

In [1927]:
sealed trait Environment 

case object EmptyEnv extends Environment
case class Extend(x: String, v: Value1, sigma: Environment) extends Environment
case class ExtendRec(f: String, x: String, e: Expr1, sigma: Environment ) extends Environment
case class ExtendMutualRec2(f1 : String, x1 : String, e1 : Expr1, f2 : String, x2 : String, e2 : Expr1, pi : Environment) extends Environment

sealed trait Value1
case class NumVal(i: Int) extends Value1
case class BinVal(b: Boolean) extends Value1
case class Closure(x: String, e: Expr1, pi: Environment) extends Value1
case object Error extends Value1

Error

In [1928]:
sealed trait Maybe[+A]
case object None extends Maybe[Nothing]
case class Some[A](v : A) extends Maybe[A]


def contains(dict : Environment, k : String ) : Boolean = {
    dict match {
        case EmptyEnv => false
        case Extend(x, v, sigma) => { if ( k == x ) {
            true
        } else {
            contains(sigma,k)
        }}
        case ExtendRec(f, x, e, sigma) => { 
            if ((k == f) || (k == x)) {
                true
            } else { 
                false
            }
        }
        case ExtendMutualRec2(f1, x1, e1, f2, x2, e2, pi) => {
            if ((k == f1) || (k == x1)) {
                true
            } else if ((k == f2) || (k == x2)) {
                true
            } else {
                false
            }
        }
    }
}

def lookupEnv(sigma: Environment, x: String): Maybe[Value1] = {
    sigma match {
        case EmptyEnv => None
        case Extend(f, v, pi) => {
            if(x == f){
                Some(v)
            } else {
                lookupEnv(pi, x)
            }
        }
        case ExtendRec(f, x, e, pi) => {
            if(x == f) {
                Some(Closure(x, e, sigma))
            } else {
                lookupEnv(pi, x)
            }
        }
        case ExtendMutualRec2(f1, x1, e1, f2, x2, e2, pi) => {
            if(x == f1){
                Some(Closure(x1,e1,sigma))
            } else if(x == f2){
                Some(Closure(x2, e2, sigma))
            } else {
                lookupEnv(pi, x)
            }
        }
    }
}
    

defined trait Maybe
defined object None
defined class Some
contains: (dict: Environment, k: String)Boolean
lookupEnv: (sigma: Environment, x: String)Maybe[Value1]


In [1929]:
val env: Environment = ExtendMutualRec2("f1", "x1", Const(0), "f2", "x2", Const(0), EmptyEnv)
passed(5)


*** Tests Passed (5 points) ***


null

In [1930]:
val env: Environment = ExtendMutualRec2("f1", "x1", Const(0), "f2", "x2", Const(0), EmptyEnv)
val f1 @ Some(Closure(_, _, pi1)) = lookupEnv(env, "f1")
val f2 @ Some(Closure(_, _, pi2)) = lookupEnv(env, "f2")
lookupEnv(pi1, "f1") == f1
lookupEnv(pi1, "f2") == f2
lookupEnv(pi2, "f2") != f1
lookupEnv(pi2, "f2") == f2
passed(5)


*** Tests Passed (5 points) ***


null

### Problem 1c (10 points): Eval

Complete the ``eval`` function that supports mutual recursion.

$$\texttt{eval} : Expr1 \rightarrow Environment \rightarrow Value1$$

In [1931]:
def eval( e : Expr1, env : Environment) : Value1 = 
    e match {
        case Const(n)        => NumVal(n) 
        case True            => BinVal(true)
        case False           => BinVal(false)
        case Ident(s)        => if (contains(env, s)) 
                                    lookupEnv(env,s) match {
                                        case Some(v) => v
                                        case None => Error
                                    }
                                else Error
        case Plus(e1, e2)    => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => NumVal( v1 + v2 )
            case _ => Error
        }
        case Minus(e1, e2)   => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => NumVal( v1 - v2 )
            case _ => Error
        }
        case Mult(e1, e2)    => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => NumVal( v1 * v2 )
            case _ => Error
        }
        case Neg(e1)         => eval(e1, env) match {
            case NumVal(v) => NumVal(-v)
            case BinVal(v) => BinVal(!v)
            case _ => Error
        }
        case And(e1, e2)     => (eval(e1,env), eval(e2,env)) match {
            case (BinVal(v1), BinVal(v2)) => BinVal( v1 && v2 )
            case _ => Error
        }
        case Or(e1, e2)      => (eval(e1,env), eval(e2,env)) match {
            case (BinVal(v1), BinVal(v2)) => BinVal( v1 || v2 )
            case _ => Error
        }
        case Geq(e1, e2)     => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => BinVal( v1 >= v2 )
            case _ => Error
        }
        case Eq(e1, e2)      => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => BinVal( v1 == v2 )
            case (BinVal(v1), BinVal(v2)) => BinVal( v1 == v2 )
            case (BinVal(_), NumVal(_)) => BinVal(false)
            case (NumVal(_), BinVal(_)) => BinVal(false)
            case _ => Error
        }
        case ITE(ec, et, ef) => eval(ec,env) match {
            case BinVal(true) => eval(et,env)
            case BinVal(false) => eval(ef,env)
            case _ => Error
        }
        case Let(name, ed, ei) => eval(ed, env) match {
            case Error => Error
            case vd => {
                val newenv = Extend(name, vd, env)
                eval(ei, newenv)
            }
        }
        
        case FunDef(arg, eb) => Closure( arg, eb, env)
        
        case FunCall(ef, ea) => (eval(ef, env), eval(ea, env)) match {
            case (Closure(x, eb, pi), va) => {
                val newenv = Extend(x, va, pi)
                eval(eb, newenv)
            }
            case _ => Error
        }
        
        case LetRec(f, x, eb, ei) => {
            val sigmaprime = ExtendRec(f, x, eb, env)
            eval( ei, sigmaprime)
        }
        case LetRec2(f1, x1, e1, f2, x2, e2, e3) => {
            val sigmaprime = ExtendMutualRec2(f1, x1, e1, f2, x2, e2, env)
            eval(e3, sigmaprime)
        }

    }

def evalProgram(p: Program1) = {
    p match { 
        case Main(e) => eval(e, EmptyEnv)
    }
}


eval: (e: Expr1, env: Environment)Value1
evalProgram: (p: Program1)Value1


In [1932]:
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val bar = Ident("bar")

val e1 = ITE( Geq(x, Const(0)), x, FunCall(bar, Plus(x, Const(1)))) // if x >= 0 then x else bar(1 + x)
val e2 = ITE( Geq(Const(1), x), Plus(Const(2), x), FunCall(foo, Minus(x, Const(2)))) // if 1 >= x then 2 + x else foo(x-2)
val e3 = FunCall(bar, Const(10))

val lr2 = LetRec2("foo", "x", e1, "bar", "x", e2, e3)
val p1 = Main(lr2)
assert(evalProgram(p1) == NumVal(8), "Test 1 of Set 1 failed")

val e4 = FunCall(foo, Const(12))
val p2 = Main(LetRec2("foo", "x", e1, "bar", "x", e2, e4))
assert(evalProgram(p2) == NumVal(12), "Test 2 of Set 1 failed")

val e5 = FunCall(foo, Const(-12))
val p3 = Main(LetRec2("foo", "x", e1, "bar", "x", e2, e5))
assert(evalProgram(p3) == NumVal(-9), "Test 3 of Set 1 failed")

val e6 = FunCall(bar, Const(-12))
val p4 = Main(LetRec2("foo", "x", e1, "bar", "x", e2, e6))
assert(evalProgram(p4) == NumVal(-10), "Test 4 of Set 1 failed")

val e7 = FunCall(bar, Const(2))
val p5 = Main(LetRec2("foo", "x", e1, "bar", "x", e2, e7))
assert(evalProgram(p5) == NumVal(0), "Test 5 of Set 1 failed")

passed(5)


*** Tests Passed (5 points) ***


null

In [1933]:
/*
let rec 
        pos = function (x) 
                 if (x >= 0)
                 then
                    if (x <= 0.5)
                     then x
                     else 0.5 + pos(x-1)
                 else neg(x)
        neg = function (y)
                 if (y <= 0)
                 then pos (1 + y * y)
                 else pos(1 - y)
   in 
     ...
*/

val x = Ident("x")
val y = Ident("y")
val pos = Ident("pos")
val neg = Ident("neg")

val e11 = ITE(Geq(Const(1), x), x, Plus(Const(1), FunCall(pos, Minus(x, Const(2)))))
val e1 = ITE(Geq(x, Const(0)), e11, FunCall(neg, x))
val e2 = ITE(Geq(Const(0), y), FunCall(pos, Plus(Const(2), Mult(y,y))), FunCall(pos, Minus(Const(2), y)))

val t1 = FunCall(neg, Const(10))
val lr1 = LetRec2("pos", "x", e1, "neg", "y", e2, t1)
val p1 = Main(lr1)


assert(evalProgram(p1) == NumVal(33), "Test 1 of set 2 failed")

val t2 = FunCall(pos, Const(-8))
val lr2 = LetRec2("pos", "x", e1, "neg", "y", e2, t2)
val p2 = Main(lr2)
assert(evalProgram(p2) == NumVal(33), "Test 2 of set 2 failed")



val t3 = FunCall(pos, Const(-1))
val lr3 = LetRec2("pos", "x", e1, "neg", "y", e2, t3)
val p3 = Main(lr3)
assert(evalProgram(p3) == NumVal(2), "Test 3 of set 2 failed")


passed(5)


*** Tests Passed (5 points) ***


null

## Problem 2 (65 Points) : Adding Mutable References to Lettuce


We will now discuss an extension of lettuce with mutable references. What are mutable references? 

- Mutable references are addresses to locations in memory that hold a value.  
  - For instance, consider memory cell 1 holds the value `NumVal(10)`.
  - A mutable reference is of the form `Ref(1)`, that tells us that this is a reference to memory address `1` and looking up address 1, we find the value `NumVal(10)` stored there.

- We will have an abstraction of memory wherein, the memory has cells numbered 0, 1, 2, ... and each cell can hold a value. This could be a number, a closure, a boolean or another reference (reference to reference!), etc..

We will augment Lettuce with three statements:
- $NewRef(\text{value})$, create a new memory cell and place `value` inside it. This returns a reference to the memory cell created.
- $DeRef(\text{reference})$, fetch the value stored in the memory cell that is referenced by `reference`.
- $AssignRef(\text{reference}, \text{value})$, change the memory cell pointed to by `reference` to have the value `value`. 

References are very much like pointers in C/C++. However, the main difference is that pointers are very low level. Here our references can store arbitrary values inside them. Therefore, unlike pointers, the size of the memory being pointed to can be variable.

- `NewRef` takes in an Expr1ession and makes a fresh reference. It them stores the value into this reference.
- `DeRef` takes in an Expr1 but as you will see, the Expr1 needs to be a reference or else we will throw an error and bail.
- `AssignRef` assigns a reference produced by evaluating its first arugment to the value from the second argument. What should assignment return? This depends on our design:


### Problem 2a (5 points) : Extend the grammar of a simple version of Lettuce to support mutable references:


$$\begin{array}{rcll}
\mathbf{Expr2} \rightarrow&  \\
 &&\ldots \\
 & | & NewRef(\mathbf{Expr2}) & \text{create a new reference and place the value of Expr2 in it.}\\
 & | & DeRef(\mathbf{Expr2}) & \text{dereference a reference and return the value}\\
 & | & AssignRef(\mathbf{Expr2}, \mathbf{Expr2}) & \text{assign a reference to a value. }
\end{array}$$

In [1934]:
sealed trait Program2
case class Main2(e: Expr2) extends Program2

sealed trait Expr2

case class Const(v: Int) extends Expr2
case class Ident(s: String) extends Expr2
case class Plus(e1: Expr2, e2: Expr2) extends Expr2
case class Minus(e1: Expr2, e2: Expr2) extends Expr2
case class Mult(e1: Expr2, e2: Expr2) extends Expr2
case class Geq(e1: Expr2, e2:Expr2) extends Expr2
case class Eq(e1: Expr2, e2: Expr2) extends Expr2
case class ITE(e: Expr2, eIf: Expr2, eElse: Expr2) extends Expr2
case class Let(s: String, defExpr2: Expr2, bodyExpr2: Expr2) extends Expr2
case class FunDef(param: String, bodyExpr2: Expr2) extends Expr2
case class FunCall(funCalled: Expr2, argExpr2: Expr2) extends Expr2
case class NewRef(e1 : Expr2) extends Expr2
case class DeRef(e1 : Expr2) extends Expr2
case class AssignRef(e1 : Expr2, e2 : Expr2) extends Expr2

defined trait Program2
defined class Main2
defined trait Expr2
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class ITE
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef


In [1935]:
val e1 = NewRef(Const(10))
val e2 = DeRef(NewRef(Const(10)))
val e3 = AssignRef(NewRef(Const(10)), Const(11))
passed(5)


*** Tests Passed (5 points) ***


null

### Problem 2b (5 points) : Extend Values to support references

References are a possible value in Lettuce. Add reference to our value type.

A reference value will look like `Reference(j)`, which is a reference to cell number j in our store. 

$$\begin{array}{rcll}
\mathbf{Value2} \rightarrow&  \\
 &&\ldots \\
 & | & Reference(Int)
\end{array}$$

In [1936]:
sealed trait Value2

case class NumVal(f: Int) extends Value2
case class BinVal(b: Boolean) extends Value2
case class Closure(x: String, e: Expr2, pi: Map[String, Value2]) extends Value2
case class Reference( x : Int) extends Value2
case object ErrorValue extends Value2

ErrorValue

In [1937]:
val newval : Value2 = Reference(10)
passed(5)


*** Tests Passed (5 points) ***


null

### Problem 2c (10 points) : Mutable Store

We need a mutable store to implement references

To strat implementing a strore, we will assume that memory is "unlimited". 
Memory address are going to be numbered 0, 1, 2, .... with natural numbers and each address is going to be associated with a value. 

The store needs to support the following operations.

- Create a new memory cell in the store and assign it to a value. This is exactly what will implement the `NewRef` operation. Let us call it ``createNewCell``. It will return the new store and the new cell number.

$$ \texttt{createNewCell} : Store \rightarrow Value \rightarrow (Store, Int) $$

- Lookup the value of a memory cell. Let us call it ``lookupCellValue``. If the value does not exist, we will return error (and bail).

$$ \texttt{lookupCellValue} : Store \rightarrow Int \rightarrow Value $$

- Assign a cell to a new value. Let us call it ``assignToCell``. This function returns the new store obtained.

$$ \texttt{assignToCell} : Store \rightarrow Int \rightarrow Value \rightarrow Store $$



In [1938]:
case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value2])



def createNewCell(s: ImmutableStore, v: Value2): (ImmutableStore, Int) = {
    s match {
        case ImmutableStore(i, m) => {
            val newInt = i + 1
            val newMap = m + (newInt -> v)
            val newStore = new ImmutableStore(newInt, newMap)
            (newStore, newInt)
        }
    }
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value2 = {
    s match {
        case ImmutableStore(i, m) => m.getOrElse(j, ErrorValue)
    }
}
    
def assignToCell(s: ImmutableStore, j: Int, v: Value2): ImmutableStore = {
    s match {
        case ImmutableStore(i, m) => {
            val newMap = m + (j -> v)
            val newStore = new ImmutableStore(i, newMap)
            newStore
        }
    }
}

defined class ImmutableStore
createNewCell: (s: ImmutableStore, v: Value2)(ImmutableStore, Int)
lookupCellValue: (s: ImmutableStore, j: Int)Value2
assignToCell: (s: ImmutableStore, j: Int, v: Value2)ImmutableStore


In [1939]:
val store0 = new ImmutableStore(0, Map())
val (store1,j) = createNewCell(store0, NumVal(10))
val store2 = assignToCell(store1, j, NumVal(20) )
val lookupold = lookupCellValue(store1, j)
val lookupnew = lookupCellValue(store2, j)

assert(lookupold == NumVal(10))
assert(lookupnew == NumVal(20))
passed(10)


*** Tests Passed (10 points) ***


null

### Problem 2d (20 points): Eval

Write the ``evalExpr2`` function that supports mutable references.

$$\texttt{evalExpr} : Expr \rightarrow Store \rightarrow Value$$

Important: Side effects happen in the order they appear in the program.

In [1940]:
def evalExpr2(e1 : Expr2, env : Map[String, Value2], store : ImmutableStore) : (Value2, ImmutableStore) = {
    e1 match {
        case Const(n) => (NumVal(n), store)
        case Ident(s) => {
            if (env.contains(s)) {
                (env(s), store)
            } else {
                (ErrorValue, store)
            }
        }
        case Plus(e1,e2) => {
            evalExpr2(e1, env, store) match {
                case (NumVal(a), store) => { 
                    evalExpr2(e2, env, store) match {
                        case (NumVal(b), store2) => (NumVal(a + b), store2)
                        case _ => (ErrorValue, store2)
                    }
                }
                case _ => (ErrorValue, store)
            }
        }
        case Minus(e1,e2) => {
             evalExpr2(e1, env, store) match {
                case (NumVal(a), store) => { 
                    evalExpr2(e2, env, store) match {
                        case (NumVal(b), store2) => (NumVal(a - b), store2)
                        case _ => (ErrorValue, store2)
                    }
                }
                case _ => (ErrorValue, store)
            }
        }
        case Mult(e1,e2) => {
             evalExpr2(e1, env, store) match {
                case (NumVal(a), store) => { 
                    evalExpr2(e2, env, store) match {
                        case (NumVal(b), store2) => (NumVal(a * b), store2)
                        case _ => (ErrorValue, store2)
                    }
                }
                case _ => (ErrorValue, store)
            }
        }
        case Geq(e1,e2) => {
            evalExpr2(e1, env, store) match {
                case (NumVal(a), store) => { 
                    evalExpr2(e2, env, store) match {
                        case (NumVal(b), store2) => (BinVal(a >= b), store2)
                        case _ => (ErrorValue, store2)
                    }
                }
                case _ => (ErrorValue, store)
            }
        }
        case Eq(e1,e2) => {
            val (a, b) = evalExpr2(e1, env, store)
            val (c, d) = evalExpr2(e2, env, b)
            (a,c) match {
                case (NumVal(x), NumVal(y)) => {
                    if(x == y){
                        (BinVal(true), d)
                    } else {
                        (BinVal(false), d)
                    }
                }
                case (BinVal(n), BinVal(m)) => {
                    if(n == m){
                        (BinVal(true), d)
                    } else {
                        (BinVal(false), d)
                    }
                }
                case _ => (BinVal(false), d)
            }
        }
        case ITE(ec, et, ef) => {
            evalExpr2(ec, env, store) match {
                case (BinVal(true), store) => evalExpr2(et, env, store)
                case (BinVal(false), store) => evalExpr2(ef, env, store)
                case (_ , store) => (ErrorValue, store)
            }
        }
        case Let(name, ed, ei) => {
            evalExpr2(ed, env, store) match {
                case (ErrorValue, store) => (ErrorValue, store)
                case (a, store) => {
                    val newenv = env + (name -> a)
                    evalExpr2(ei, newenv, store)
                }
            }
        }
// 244
        case FunDef(arg, eb) => {
            (Closure(arg, eb, env), store)
        }
        case FunCall(ef , ea) => {
            evalExpr2(ef, env, store) match {
                case (Closure(x, eb, pi), store) => {
                    evalExpr2(ea, env, store) match {
                        case (a, s2) => {
                            val newenv = pi + (x -> a)
                            evalExpr2(eb, newenv, s2)
                        } 
                    }
                }
                case (_, store) => (ErrorValue, store)
            }
        }
// 261
        case NewRef(e1) => {
            evalExpr2(e1, env, store) match {
                case(x : Value2, store) => {
                    createNewCell(store, x) match {
                        case (s2, position) => (Reference(position), s2)
                    }
                }
                case (_, s) => (ErrorValue, s)
            }
        }
        case DeRef(e1) => {
            evalExpr2(e1, env, store) match {
                case (Reference(n), store) => {
                    lookupCellValue(store, n) match {
                        case v => (v, store)
                    }
                }
                case (_, s) => {
                    (ErrorValue, s)
                }
            }
        }
        case AssignRef(e1, e2) => {
            evalExpr2(e1, env, store) match {
                case (Reference(n), store) => {
                    evalExpr2(e2, env, store) match {
                        case (v, s2) => (v, assignToCell(s2, n, v))
                    }
                }
                case (_, s) => {
                    (ErrorValue, s)
                }
            }
        }
    }
}

def evalProgram2(p: Program2) = p match {
    case Main2(e) => { 
        val (v1, s1) = evalExpr2(e, Map(), new ImmutableStore(0, Map()))
        v1
    }
}

evalExpr2: (e1: Expr2, env: Map[String,Value2], store: ImmutableStore)(Value2, ImmutableStore)
evalProgram2: (p: Program2)Value2


In [1941]:
val z = Ident("z")
val x = Ident("x")
val y = Ident("y")

val e1 = Let("z", AssignRef(x, y), z)
val e2 = Let("y", Plus(Const(1), DeRef(x)), e1)
val e = Let("x", NewRef(Const(10)), e2)
val prog = Main2(e)

assert(evalProgram2(prog) == NumVal(11))
passed(5)


*** Tests Passed (5 points) ***


null

In [1942]:
val incr = Ident("incr")
val decr = Ident("decr")
val zero = Ident("zero")
val r = Ident("r")
val x = Ident("x")
val e1 = Let("dummy3", FunCall(decr, r), Ident("dummy3"))
val e2 = Let("dummy2", FunCall(incr, r),  e1)
val e3 = Let("dummy1", FunCall(incr, r), e2)
val e4 = Let("dummy0", FunCall(zero, r), e3)
val rr = Let("r", NewRef(Const(15)), e4)
val z = Let("zero", FunDef("x", AssignRef(x, Const(0))), rr)
val d = Let("decr", FunDef("x", AssignRef(x, Minus(DeRef(x), Const(1)))), z)
val i = Let("incr",FunDef("x", AssignRef(x, Plus(DeRef(x), Const(1)))), d )
val prog = Main2(i)

assert(evalProgram2(prog) == NumVal(1))
passed(5)


*** Tests Passed (5 points) ***


null

In [1943]:
val r = Ident("r")
val x = Ident("x")
val f = Ident("f")

val e1 = Minus(FunCall(f, r), FunCall(f, r))
val e2 = Let("r", NewRef(Const(10)), e1)
val fbody = Let("tmp", DeRef(x), Let("dummy", AssignRef(x, Const(25)), Ident("tmp")))
val fdef = FunDef("x", fbody)
val e3 = Let("f", fdef, e2)
val prog = Main2(e3)

assert(evalProgram2(prog) == NumVal(-15))
passed(5)


*** Tests Passed (5 points) ***


null

In [1944]:
/*
let x = NewRef(10) in 
   let g = function (y) 
            DeRef(x)
        in 
       let dummy = AssignRef(x, 20) in 
            g (dummy)
*/
val dummy = Ident("dummy")
val x = Ident("x")
val g = Ident("g")
val e1 = FunCall(g, dummy)
val e2 = Let("dummy", AssignRef(x, Const(20)), e1)
val fdef = FunDef("y", DeRef(x))
val e3 = Let("g", fdef, e2)
val e4 = Let("x", NewRef(Const(10)), e3)
val prog = Main2(e4)

assert(evalProgram2(prog) == NumVal(20))
passed(5)


*** Tests Passed (5 points) ***


null

### Problem 2e (5 points) : Ref Types

Extend the types to allow for reference types.

$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType & \leftarrow \text{written as } \ num \\
& | & BoolType & \leftarrow \text{written as } \ bool \\
& | & FunType(\mathbf{Type} , \mathbf{Type} ) & \leftarrow \text{written as } \ t_1 \Rightarrow t_2 \\
& | & RefType(\mathbf{Type}) & \leftarrow \text{written as } \ ref(t_1) \\
\end{array}$$

In [1945]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(t1: Type, t2: Type) extends Type
case class RefType(t1: Type) extends Type
case object TypeError extends Type

TypeError

In [1946]:
val tref1 = RefType(NumType)
val tref2 = RefType(RefType(BoolType))
passed(5)


*** Tests Passed (5 points) ***


null

### Problem 2f (20 points ) : Typechecking Refs

As an example consider the well typed and annotated program:

~~~
let x: num = 25 in 
    let y : ref(num) = NewRef( - x ) in 
     let f : ref(ref(num) => bool) = NewRef( function (z: ref(num)) DeRef(z) - x >= 45 ) in 
          let z : ref(bool) = DeRef(f)(y)
~~~

Extend the typecheck program below to support typechecking for references.

In [1947]:


sealed trait Program3
sealed trait Expr3

case class Const(f: Double) extends Expr3
case class Ident(s: String) extends Expr3
case class Plus(e1: Expr3, e2: Expr3) extends Expr3
case class Minus(e1: Expr3, e2: Expr3) extends Expr3
case class Geq(e1: Expr3, e2: Expr3) extends Expr3
case class IfThenElse(e1: Expr3, e2: Expr3, e3: Expr3) extends Expr3
case class Let(x: String, xType: Type, e1: Expr3, e2: Expr3) extends Expr3
case class FunDef(id: String, idType: Type, e: Expr3) extends Expr3
case class FunCall(calledFun: Expr3, argExpr3: Expr3) extends Expr3
case class NewRef(e: Expr3) extends Expr3
case class DeRef(e: Expr3) extends Expr3
case class AssignRef(e1: Expr3, e2: Expr3) extends Expr3
case class Main(e: Expr3) extends Program3

def typeEquals(t1: Type, t2: Type): Boolean = t1 == t2
case class TypeErrorException(s: String) extends Exception


defined trait Program3
defined trait Expr3
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef
defined class Main
typeEquals: (t1: Type, t2: Type)Boolean
defined class TypeErrorException


In [1948]:
def typeOf(e: Expr3, alpha: Map[String, Type]): Type = {
    def checkType(opName: String, e1: Expr3, t1: Type, e2: Expr3, t2: Type, resType: Type): Type = {
        val t1hat = typeOf(e1, alpha)
        if (! typeEquals(t1hat, t1)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t1, obtained $t1hat")
        }
        
        val t2hat = typeOf(e2, alpha)
        if (! typeEquals(t2hat, t2)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t2, obtained $t2hat")
        }
        
        resType
    }
    
    e match {
        case Const(f) => NumType
        case Ident(s) => {if (alpha contains s)
                             alpha(s)
                          else 
                             throw TypeErrorException(s"Unknown identifier $s")}
        case Plus(e1, e2) =>  checkType("Plus", e1,  NumType, e2, NumType, NumType)
        case Minus(e1, e2) => checkType("Minus",e1,  NumType, e2, NumType, NumType)
        case Geq(e1, e2) => checkType("Geq", e1,  NumType, e2, NumType, BoolType)
        case IfThenElse(e, e1, e2) => {
            val t = typeOf(e, alpha)
            if (t == BoolType){
                val t1 = typeOf(e1, alpha)
                val t2 = typeOf(e2, alpha)
                if (typeEquals(t1, t2))
                    t1
                else 
                    throw TypeErrorException(s"If then else returns unequal types $t1 and $t2")
            } else {
                throw TypeErrorException(s"If then else condition expression not boolean $t")
            }
        }

        case Let(x, t, e1, e2) => {
            val t1 = typeOf(e1, alpha)
            if (typeEquals(t1, t)){
                val newAlpha = alpha + (x -> t)
                typeOf(e2, newAlpha)
            } else {
                throw TypeErrorException(s"Let binding has type $t whereas it is bound to expression of type $t1")
            }
        }

        case FunDef(x, t1, e) => {
            val newAlpha = alpha + (x -> t1)
            val t2 = typeOf(e, newAlpha)
            FunType(t1, t2)
        }
// 195
        case FunCall(e1, e2) => {
            val ftype = typeOf(e1, alpha)
            ftype match {
                case FunType(t1, t2) => {
                    val argType = typeOf(e2, alpha)
                    if (typeEquals(argType, t1)){
                        t2
                    } else {
                        throw TypeErrorException(s"Call to function with incompatible argument type. Expected $t1, obtained $argType")
                    }
                }
                case _ => { throw TypeErrorException(s"Call to function but with a non function type $ftype")}

            }
        }

        case NewRef(e) => {
            val reftype = typeOf(e, alpha)
            reftype match {
                case TypeError => { throw TypeErrorException(s"TypeError")}
                case FunType(NumType,NumType) => { throw TypeErrorException(s"TypeError")}
                case a => RefType(reftype)
            }
        }

        case AssignRef(e1, e2) => {
            e1 match {
                case Ident(n) => {
                    val t2 = typeOf(e2, alpha)
                    val newalpha = alpha + (n -> t2)
                    val tr = typeOf(e2, newalpha)
                    tr
                }
                case _ => { throw TypeErrorException(s"TypeError")}
            }
        }
        case DeRef(e) => {
            e match {
                case Ident(s) => {
                    if (alpha.contains(s)){
                        val t1 = typeOf(e, alpha)
                        t1 match {
                            case RefType(a) => a
                            case _ => { throw TypeErrorException(s"TypeError")}
                        }
                    } else {
                            throw TypeErrorException(s"TypeError")
                        }
                }
                case _ => throw TypeErrorException(s"TypeError")
            }
        }
    }
}

def typeOfProgram(p: Program3) = p match {
    case Main(e) => {
            val t = typeOf(e, Map())
            println(s"Program type computed successfully as $t")
            t
    }
}


typeOf: (e: Expr3, alpha: Map[String,Type])Type
typeOfProgram: (p: Program3)Type


In [1949]:
//BEGIN TEST

/* 
let x : ref(num) = NewRef(10 ) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */

val p1 = Let("x", RefType(NumType), NewRef(Const(10)), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t1 = typeOfProgram(Main(p1))
assert(t1 == NumType, "Test 1 failed: answer should be NumType")
passed(4)
//END TEST

Program type computed successfully as NumType

*** Tests Passed (4 points) ***


null

In [1950]:
//BEGIN TEST
/* 
let x : ref(num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p2 = Let("x", RefType(NumType), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t2 = try {
   typeOfProgram(Main(p2))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(4)
//END TEST


*** Tests Passed (4 points) ***


null

In [1951]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p3 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t3 = try {
   typeOfProgram(Main(p3))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(4)
//END TEST


*** Tests Passed (4 points) ***


null

In [1952]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(NewRef(35), 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p4 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(NewRef(Const(35)), Const(30) ), DeRef(Ident("x"))) )
val t4 =  typeOfProgram(Main(p4))
assert(t4 == FunType(NumType, NumType), "Test failed")
passed(4)
//END TEST

TypeErrorException: TypeErrorException

In [1953]:
//BEGIN TEST

val y = Ident("y")
val x = Ident("x")
val z = Ident("z")
val t1 = RefType(NumType)
val t2 = FunType(t1, NumType)
val t3 = RefType(t2)

val v1 = FunCall(DeRef(y), x)
val v2 = Let("z", RefType(BoolType), NewRef(Geq(DeRef(x), Const(25))), v1)
val fdef = FunDef("z", RefType(NumType), Plus(DeRef(z), DeRef(x)))
val v3 = Let("y", t3, NewRef(fdef), v2)
val v4 = Let("x", t1, NewRef(Const(10)), v3)

val p5 = Main(v4)
assert(typeOfProgram(p5) == NumType, "Test failed")
passed(4)

//END TEST

Program type computed successfully as NumType

*** Tests Passed (4 points) ***


null